Código Completo com Detecção de Pessoas Usando YOLOv3

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import tkinter as tk
from PIL import Image, ImageTk
import os
print(os.listdir('.'))  # Liste arquivos no diretório atual

# Caminho completo para os arquivos
cfg_file = 'C:/Users/lucas/OneDrive/Pós_Graduação/7- Desenvolvimento de aplicações IA – robótica, imagem e visão computacional/TEMA 04 - MÉTODOS INTELIGENTES EM PROCESSAMENTO DE IMAGENS/Teoria em pratica/yolov3.cfg'
weights_file = 'C:/Users/lucas/OneDrive/Pós_Graduação/7- Desenvolvimento de aplicações IA – robótica, imagem e visão computacional/TEMA 04 - MÉTODOS INTELIGENTES EM PROCESSAMENTO DE IMAGENS/Teoria em pratica/yolov3.weights'

# Carregar o modelo YOLOv3
net = cv2.dnn.readNet(weights_file, cfg_file)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Função para detectar pessoas
def detect_people(frame):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 0:  # class_id 0 é 'person' no COCO dataset
                    center_x = int(obj[0] * width)
                    center_y = int(obj[1] * height)
                    w = int(obj[2] * width)
                    h = int(obj[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return indices, boxes, confidences

# Função para capturar um frame da câmera
def get_frame_from_camera(camera_url):
    cap = cv2.VideoCapture(camera_url)
    ret, frame = cap.read()
    cap.release()
    return frame if ret else None

# Função para atualizar a interface gráfica
def update_gui():
    frame = get_frame_from_camera(camera_url)
    if frame is not None:
        indices, boxes, confidences = detect_people(frame)
        
        # Desenhar as caixas delimitadoras para as pessoas detectadas
        for i in indices:
            i = i[0]
            x, y, w, h = boxes[i]
            person_frame = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'Person', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Reconhecer comportamento para a pessoa detectada
            if person_frame.size > 0:
                person_frame_rgb = cv2.cvtColor(person_frame, cv2.COLOR_BGR2RGB)
                predictions = recognize_behavior(person_frame_rgb)
                prediction_text = "\n".join([f'{label}: {score:.2f}' for (_, label, score) in predictions])
                cv2.putText(frame, prediction_text, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Converter a imagem para o formato Tkinter
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame_rgb)
        image_tk = ImageTk.PhotoImage(image=image)

        # Atualizar a imagem na interface
        panel.config(image=image_tk)
        panel.image = image_tk

        # Atualizar o texto com o número de pessoas detectadas
        prediction_text = f'Número de pessoas detectadas: {len(indices)}'
        prediction_label.config(text=prediction_text)
    
    root.after(1000, update_gui)  # Atualizar a cada 1 segundo

# Configurações da interface gráfica
root = tk.Tk()
root.title("Sistema de Vigilância")

panel = tk.Label(root)
panel.pack()

prediction_label = tk.Label(root, text="Reconhecimento de Comportamento")
prediction_label.pack()

camera_url = 'http://ip_da_camera:porta/video'
update_gui()  # Iniciar a atualização da interface gráfica

root.mainloop()


['Análise_de_comportamentos_suspeitos.ipynb', 'yolov3.cfg', 'yolov3.weights']


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: C:/Users/lucas/OneDrive/Pós_Graduação/7- Desenvolvimento de aplicações IA – robótica, imagem e visão computacional/TEMA 04 - MÉTODOS INTELIGENTES EM PROCESSAMENTO DE IMAGENS/Teoria em pratica/yolov3.cfg in function 'cv::dnn::dnn4_v20240521::readNetFromDarknet'
